# Filtering Microcap Alts

In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep


# Functions for pulling from Coinmarketcap
- coindfpull: pulls coin information from cmc including total coin supply
- tokendfpull: pulls token information from cmc including total token supply
- coincircdf: pulls coin circulating supply from cmc
- tokencircdf: pulls token circulating supply from cmc

In [2]:
def coindfpull(pagenum):
    site = 'https://coinmarketcap.com/coins/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[9].split('data-btc="')[1].split(' " data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[9].split('data-btc="')[1].split(' " data-usd="')[0]]
            pricesusd = pricesusd + [columns[15].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            pricesbtc = pricesbtc + [columns[15].split('" data-btc="')[1].split('" data-usd="')[0]]
            volumes24husd = volumes24husd + [columns[18].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            volumes24hbtc = volumes24hbtc + [columns[18].split('" data-btc="')[1].split('" data-usd="')[0]]
            totalsupplies = totalsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def tokendfpull(pagenum):
    site = 'https://coinmarketcap.com/tokens/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    platforms = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            platforms = platforms + [columns[0].split('data-platformsymbol="')[1].split('" ')[0]]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[10].split('data-btc="')[1].split(' " ')[1].split('data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[10].split('data-btc="')[1].split(' " ')[0]]
            pricesusd = pricesusd + [columns[16].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            pricesbtc = pricesbtc + [columns[16].split('data-btc="')[1].split('" ')[0]]
            volumes24husd = volumes24husd + [columns[19].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            volumes24hbtc = volumes24hbtc + [columns[19].split('data-btc="')[1].split('" ')[0]]
            totalsupplies = totalsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0].split('" ')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "Platform": platforms,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def coincircdf(pagenum):
    site = 'https://coinmarketcap.com/coins/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

def tokencirdf(pagenum):
    site = 'https://coinmarketcap.com/tokens/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

# Alternative Function for Pulling all Coin/Token Caps

In [17]:
def pullfulldf():
    site = 'https://coinmarketcap.com/all/views/all/'
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")

    table_start = '<table class="table js-summary-table dataTable no-footer" id="currencies-all" style="font-size: 14px; width: 100%;" role="grid">'
    table_end = '</table>'

    table = soup.find_all('table')[0]

    symbol = []
    cname = []
    mcapsusd = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    totalsupplies = []
    availablesupplies = []
    maxsupplies = []
    change1h = []
    change24h = []
    change7d = []
    i = 1
    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if not('class' in columns[0]):
            continue

        cmcname = columns[0].split('"id-')[1].replace('">','')

        apisite = 'https://api.coinmarketcap.com/v1/ticker/' + cmcname + '/'
        response = requests.get(apisite)
        json_data = json.loads(response.text)

        cname = cname + [cmcname]
        symbol = symbol + [json_data[0]['symbol']]
        mcapsusd = mcapsusd + [json_data[0]['market_cap_usd']]
        pricesusd = pricesusd + [json_data[0]['price_usd']]
        pricesbtc = pricesbtc + [json_data[0]['price_btc']]
        volumes24husd = volumes24husd + [json_data[0]['24h_volume_usd']]
        totalsupplies = totalsupplies + [json_data[0]['total_supply']]
        availablesupplies = availablesupplies + [json_data[0]['available_supply']]
        maxsupplies = maxsupplies + [json_data[0]['max_supply']]
        change1h = change1h + [json_data[0]['percent_change_1h']]
        change24h = change24h + [json_data[0]['percent_change_24h']]
        change7d = change7d + [json_data[0]['percent_change_7d']]
        
        print('Number: ' + str(i) + ' Coin: ' + json_data[0]['symbol'])
        sleep(1)
        i += 1

    fulldict = {'CMCname': cname,
                'Symbol': symbol,
                'Mcap': mcapsusd,
                'USDPrice': pricesusd,
                'BTCPrice': pricesbtc,           
                'Vol24h': volumes24husd,           
                'TotalSupply': totalsupplies,           
                'AvailableSupply': availablesupplies,    
                'MaxSupply': maxsupplies,
                'Change1h': change1h,
                'Change24h': change24h,
                'Change7d': change7d}
    fulldf = pd.DataFrame(fulldict,index = cname)
    
    return fulldf


# Creating dataframe for initial filtering
- Keep only marketcaps with less than 100M
- Remove tokens with 24 hour volumes less than 2% of marketcaps
- Remove tokens with circulating supply/total supply with less than 20%

In [10]:
# Creating DF for Coins

def microfilter1(mcapthreshold, volthreshold, supplythreshold):
    pagenmbr = 1
    coinlist = []
    while type(coindfpull(pagenmbr)) != str:
        coinlist = coinlist + [coindfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    coincirclist = []
    while type(coincircdf(pagenmbr)) != str:
        coincirclist = coincirclist + [coincircdf(pagenmbr)]
        pagenmbr += 1

    coinscirc = pd.concat(coincirclist, axis = 0)
    coins = pd.concat(coinlist, axis = 0)
    coins = coins[coins['MktCapUSD'] != '?']
    coins = pd.concat([coins.drop_duplicates(),coinscirc.drop_duplicates()], axis = 1, join = 'inner')

    # Creating DF for Tokens

    pagenmbr = 1
    tokenlist = []
    while type(tokendfpull(pagenmbr)) != str:
        tokenlist = tokenlist + [tokendfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    tokencirclist = []
    while type(tokencirdf(pagenmbr)) != str:
        tokencirclist = tokencirclist + [tokencirdf(pagenmbr)]
        pagenmbr += 1

    tokenscirc = pd.concat(tokencirclist, axis = 0)
    tokens = pd.concat(tokenlist, axis = 0)
    tokens = tokens[tokens['MktCapUSD'] != '?']
    tokens = pd.concat([tokens.drop_duplicates(),tokenscirc.drop_duplicates()], axis = 1, join = 'inner')

    totaldf = pd.concat([coins,tokens], axis = 0)
    totaldf = totaldf[totaldf['CirculatingSupply'] != 'None']
    totaldf = totaldf[totaldf['Vol24USD'] != 'None']

    totaldf['MktCapUSD'] = pd.to_numeric(totaldf['MktCapUSD'])
    totaldf['CirculatingSupply'] = pd.to_numeric(totaldf['CirculatingSupply'])
    totaldf['TotalSupply'] = pd.to_numeric(totaldf['TotalSupply'])
    totaldf['Vol24USD'] = pd.to_numeric(totaldf['Vol24USD'])
    totaldf['VolRatio'] = totaldf['Vol24USD']/totaldf['MktCapUSD']
    totaldf['SupplyRatio'] = totaldf['CirculatingSupply']/totaldf['TotalSupply']
    
    totaldf = totaldf[totaldf['MktCapUSD'] <= mcapthreshold]
    totaldf = totaldf[totaldf['VolRatio'] >= volthreshold]
    totaldf = totaldf[totaldf['SupplyRatio'] >= supplythreshold]
    
    
    return totaldf

In [11]:
def checkannouncement(cmcname):

    site = 'https://coinmarketcap.com/currencies/' + cmcname + '/'

    res = requests.get(site)
    soup = str(bs(res.text, 'lxml'))

    if "Announcement" in soup:
        announcementsite = soup.split('<li><span class="glyphicon glyphicon-bullhorn text-gray" title="Announcement">')[1].split('<a href="')[1].split('" target')[0]
    else:
        announcementsite = "NA"

    if 'bitcointalk.org' in announcementsite:
        return 1, announcementsite
    else:
        return 0, 'NA'
    
def bctcheck(cmcname):

    annsite = checkannouncement(cmcname)[1]
    parsesoup = annsite.split('=')[0] + '=' + annsite.split('=')[1].split('.')[0]
    res = requests.get(parsesoup)
    soup = str(bs(res.text, 'lxml'))
    discussion = soup.split('<td class="middletext"')[1].split('<span class="prevnext">')[0]
    discussion = discussion.split('<div class="nav"')[0]
    if 'All' in discussion.split(parsesoup)[-1]:
        lastdiscuss = discussion.split(parsesoup)[-2]
    else:
        lastdiscuss = discussion.split(parsesoup)[-1]
        
    if '<td align="right" style="padding-right:' in discussion:
        lastpgIndex = lastdiscuss.split('">')[0]
    else:
        lastpgIndex = lastdiscuss.split('">')[0]

    lastpg = parsesoup + lastpgIndex
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    totpages = soup.split('<td class="middletext"')[1].split('[<b>')[1].split('</b>]')[0]

    forumparse = lastpg.split('=')[1]
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    lastmsg = soup.split(forumparse)[1].split('<tbody>')[-1].split('<div class="smalltext">')[-1].split('</div>')[0]
    
    return totpages, lastmsg

In [ ]:
'''
# Using old method to build dataframe, no change fields available

cmcdf = microfilter1(100000000, 0.02, 0.1)

names = list(cmcdf.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        totpgs, last = bctcheck(cmcname)
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcdf,bctdf],axis = 1)
filterdf.to_csv('cmc2.csv')

'''

In [46]:
cmcfull = pullfulldf()

names = list(cmcfull.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        try:
            totpgs, last = bctcheck(cmcname)
        except:
            totpgs, last = (0,0)
        
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcfull,bctdf],axis = 1)

binance = pd.read_csv('binancelist.csv')
binance.index = binance['Name']
del binance['Ticker'], binance['Name']

bittrex = pd.read_excel('bittrexlist.xlsx')
bittrex.index = bittrex['Name']
del bittrex['Ticker'], bittrex['Name']

kucoin = pd.read_csv('kucoinlist.csv')
kucoin.index = kucoin['Name']
del kucoin['Ticker'], kucoin['Name']

cryptopia = pd.read_csv('cryptopialist.csv')
cryptopia.index = cryptopia['Name']
del cryptopia['Ticker'], cryptopia['Name']

fulldf = pd.concat([filterdf,cryptopia,kucoin,binance,bittrex],axis = 1)
fulldf.head()

fulldf.to_csv('cmc.csv')

Number: 1 Coin: BTC
Number: 2 Coin: ETH
Number: 3 Coin: XRP
Number: 4 Coin: BCH
Number: 5 Coin: ADA
Number: 6 Coin: XEM
Number: 7 Coin: LTC
Number: 8 Coin: XLM
Number: 9 Coin: MIOTA
Number: 10 Coin: DASH
Number: 11 Coin: EOS
Number: 12 Coin: NEO
Number: 13 Coin: TRX
Number: 14 Coin: XMR
Number: 15 Coin: ICX
Number: 16 Coin: QTUM
Number: 17 Coin: BTG
Number: 18 Coin: ETC
Number: 19 Coin: LSK
Number: 20 Coin: XRB
Number: 21 Coin: OMG
Number: 22 Coin: XVG
Number: 23 Coin: BNB
Number: 24 Coin: ZEC
Number: 25 Coin: BCN
Number: 26 Coin: SC
Number: 27 Coin: BCC
Number: 28 Coin: PPT
Number: 29 Coin: STRAT
Number: 30 Coin: KCS
Number: 31 Coin: SNT
Number: 32 Coin: VEN
Number: 33 Coin: BTS
Number: 34 Coin: SMART
Number: 35 Coin: ARDR
Number: 36 Coin: DOGE
Number: 37 Coin: USDT
Number: 38 Coin: STEEM
Number: 39 Coin: DCN
Number: 40 Coin: REP
Number: 41 Coin: WAVES
Number: 42 Coin: ZRX
Number: 43 Coin: DRGN
Number: 44 Coin: DGB
Number: 45 Coin: LRC
Number: 46 Coin: KMD
Number: 47 Coin: ARK
Number:

Number: 372 Coin: RC
Number: 373 Coin: BSD
Number: 374 Coin: WCT
Number: 375 Coin: XLR
Number: 376 Coin: ALQO
Number: 377 Coin: SEND
Number: 378 Coin: ESP
Number: 379 Coin: FYP
Number: 380 Coin: ICOS
Number: 381 Coin: SPF
Number: 382 Coin: BCY
Number: 383 Coin: XAUR
Number: 384 Coin: GEO
Number: 385 Coin: CSNO
Number: 386 Coin: WISH
Number: 387 Coin: IOP
Number: 388 Coin: REX
Number: 389 Coin: BITUSD
Number: 390 Coin: ADST
Number: 391 Coin: PST
Number: 392 Coin: GBX
Number: 393 Coin: UFR
Number: 394 Coin: SPR
Number: 395 Coin: ZOI
Number: 396 Coin: XPM
Number: 397 Coin: XVC
Number: 398 Coin: NVST
Number: 399 Coin: RMC
Number: 400 Coin: BTM
Number: 401 Coin: CRB
Number: 402 Coin: FRST
Number: 403 Coin: NTRN
Number: 404 Coin: OBITS
Number: 405 Coin: CURE
Number: 406 Coin: UNO
Number: 407 Coin: SUMO
Number: 408 Coin: CRC
Number: 409 Coin: ATMS
Number: 410 Coin: BON
Number: 411 Coin: CVCOIN
Number: 412 Coin: PLU
Number: 413 Coin: LEO
Number: 414 Coin: QWARK
Number: 415 Coin: INN
Number: 41

Number: 737 Coin: ONX
Number: 738 Coin: UNI
Number: 739 Coin: USC
Number: 740 Coin: XCT
Number: 741 Coin: EOT
Number: 742 Coin: BAS
Number: 743 Coin: EVIL
Number: 744 Coin: DSH
Number: 745 Coin: CCN
Number: 746 Coin: BUCKS
Number: 747 Coin: BTCR
Number: 748 Coin: VLT
Number: 749 Coin: LTB
Number: 750 Coin: XHI
Number: 751 Coin: VIDZ
Number: 752 Coin: LBTC
Number: 753 Coin: XPY
Number: 754 Coin: CNO
Number: 755 Coin: XRA
Number: 756 Coin: MNC
Number: 757 Coin: TGC
Number: 758 Coin: UNY
Number: 759 Coin: 888
Number: 760 Coin: SLG
Number: 761 Coin: B@
Number: 762 Coin: PCOIN
Number: 763 Coin: LCP
Number: 764 Coin: HERO
Number: 765 Coin: GRIM
Number: 766 Coin: FNC
Number: 767 Coin: DDF
Number: 768 Coin: TAG
Number: 769 Coin: AMBER
Number: 770 Coin: MOJO
Number: 771 Coin: UNIC
Number: 772 Coin: MNM
Number: 773 Coin: RMC
Number: 774 Coin: SDRN
Number: 775 Coin: BERN
Number: 776 Coin: SOON
Number: 777 Coin: METAL
Number: 778 Coin: FUZZ
Number: 779 Coin: KED
Number: 780 Coin: TSE
Number: 781 C

Number: 1095 Coin: MDS
Number: 1096 Coin: SHOW
Number: 1097 Coin: BIX
Number: 1098 Coin: GNX
Number: 1099 Coin: SMT
Number: 1100 Coin: BCX
Number: 1101 Coin: AIDOC
Number: 1102 Coin: BCD
Number: 1103 Coin: GTC
Number: 1104 Coin: TSL
Number: 1105 Coin: AIX
Number: 1106 Coin: CFUN
Number: 1107 Coin: KCASH
Number: 1108 Coin: QUN
Number: 1109 Coin: TIO
Number: 1110 Coin: QLC
Number: 1111 Coin: TOPC
Number: 1112 Coin: HTML
Number: 1113 Coin: AMM
Number: 1114 Coin: CAN
Number: 1115 Coin: VIU
Number: 1116 Coin: SBTC
Number: 1117 Coin: CAT
Number: 1118 Coin: ACE
Number: 1119 Coin: SPHTX
Number: 1120 Coin: DEW
Number: 1121 Coin: HPY
Number: 1122 Coin: MDT
Number: 1123 Coin: VZT
Number: 1124 Coin: MOT
Number: 1125 Coin: QBT
Number: 1126 Coin: BIG
Number: 1127 Coin: IGNIS
Number: 1128 Coin: KBR
Number: 1129 Coin: FAIR
Number: 1130 Coin: FRGC
Number: 1131 Coin: PRO
Number: 1132 Coin: XIN
Number: 1133 Coin: INF
Number: 1134 Coin: NMS
Number: 1135 Coin: UGT
Number: 1136 Coin: XTZ
Number: 1137 Coin: 

Name: 18 Alt: ethereum-classic
Name: 19 Alt: lisk
Name: 20 Alt: raiblocks
Name: 21 Alt: omisego
Name: 22 Alt: verge
Name: 23 Alt: binance-coin
Name: 24 Alt: zcash
Name: 25 Alt: bytecoin-bcn
Name: 26 Alt: siacoin
Name: 27 Alt: bitconnect
Name: 28 Alt: populous
Name: 29 Alt: stratis
Name: 30 Alt: kucoin-shares
Name: 31 Alt: status
Name: 32 Alt: vechain
Name: 33 Alt: bitshares
Name: 34 Alt: smartcash
Name: 35 Alt: ardor
Name: 36 Alt: dogecoin
Name: 37 Alt: tether
Name: 38 Alt: steem
Name: 39 Alt: dentacoin
Name: 40 Alt: augur
Name: 41 Alt: waves
Name: 42 Alt: 0x
Name: 43 Alt: dragonchain
Name: 44 Alt: digibyte
Name: 45 Alt: loopring
Name: 46 Alt: komodo
Name: 47 Alt: ark
Name: 48 Alt: veritaseum
Name: 49 Alt: wax
Name: 50 Alt: hshare
Name: 51 Alt: decred
Name: 52 Alt: electroneum
Name: 53 Alt: golem-network-tokens
Name: 54 Alt: dent
Name: 55 Alt: basic-attention-token
Name: 56 Alt: pivx
Name: 57 Alt: qash
Name: 58 Alt: salt
Name: 59 Alt: bytom
Name: 60 Alt: medibloc
Name: 61 Alt: byteball

Name: 363 Alt: circuits-of-value
Name: 364 Alt: artbyte
Name: 365 Alt: mybit-token
Name: 366 Alt: dao-casino
Name: 367 Alt: audiocoin
Name: 368 Alt: oraclechain
Name: 369 Alt: pinkcoin
Name: 370 Alt: quantum
Name: 371 Alt: chronobank
Name: 372 Alt: russiacoin
Name: 373 Alt: bitsend
Name: 374 Alt: waves-community-token
Name: 375 Alt: solaris
Name: 376 Alt: alqo
Name: 377 Alt: social-send
Name: 378 Alt: espers
Name: 379 Alt: flypme
Name: 380 Alt: icos
Name: 381 Alt: sportyfi
Name: 382 Alt: bitcrystals
Name: 383 Alt: xaurum
Name: 384 Alt: geocoin
Name: 385 Alt: bitdice
Name: 386 Alt: mywish
Name: 387 Alt: internet-of-people
Name: 388 Alt: real-estate-tokens
Name: 389 Alt: bitusd
Name: 390 Alt: adshares
Name: 391 Alt: primas
Name: 392 Alt: gobyte
Name: 393 Alt: upfiring
Name: 394 Alt: spreadcoin
Name: 395 Alt: zoin
Name: 396 Alt: primecoin
Name: 397 Alt: vcash
Name: 398 Alt: nvo
Name: 399 Alt: russian-mining-coin
Name: 400 Alt: bitmark
Name: 401 Alt: creditbit
Name: 402 Alt: firstcoin
Name

Name: 695 Alt: 42-coin
Name: 696 Alt: pakcoin
Name: 697 Alt: bitstar
Name: 698 Alt: bitcoin-red
Name: 699 Alt: adcoin
Name: 700 Alt: eternity
Name: 701 Alt: zcash-gold
Name: 702 Alt: pascal-lite
Name: 703 Alt: maxcoin
Name: 704 Alt: prototanium
Name: 705 Alt: opal
Name: 706 Alt: happycoin
Name: 707 Alt: triangles
Name: 708 Alt: giga-watt-token
Name: 709 Alt: cryptocarbon
Name: 710 Alt: britcoin
Name: 711 Alt: ammo-rewards
Name: 712 Alt: altcommunity-coin
Name: 713 Alt: platinumbar
Name: 714 Alt: whalecoin
Name: 715 Alt: postoken
Name: 716 Alt: truckcoin
Name: 717 Alt: 8bit
Name: 718 Alt: dix-asset
Name: 719 Alt: supercoin
Name: 720 Alt: biblepay
Name: 721 Alt: coin2-1
Name: 722 Alt: kushcoin
Name: 723 Alt: yenten
Name: 724 Alt: signatum
Name: 725 Alt: 808coin
Name: 726 Alt: colossuscoin-v2
Name: 727 Alt: redcoin
Name: 728 Alt: blakestar
Name: 729 Alt: monkey-project
Name: 730 Alt: bitcoal
Name: 731 Alt: phoenixcoin
Name: 732 Alt: trident
Name: 733 Alt: goldblocks
Name: 734 Alt: ethereu

Name: 1020 Alt: cypher
Name: 1021 Alt: yacoin
Name: 1022 Alt: sacoin
Name: 1023 Alt: bitz
Name: 1024 Alt: usde
Name: 1025 Alt: wmcoin
Name: 1026 Alt: franko
Name: 1027 Alt: firecoin
Name: 1028 Alt: bitgold
Name: 1029 Alt: islacoin
Name: 1030 Alt: ether-for-the-rest-of-the-world
Name: 1031 Alt: coexistcoin
Name: 1032 Alt: javascript-token
Name: 1033 Alt: crypto
Name: 1034 Alt: sling
Name: 1035 Alt: warp
Name: 1036 Alt: piecoin
Name: 1037 Alt: guccionecoin
Name: 1038 Alt: dollarcoin
Name: 1039 Alt: pulse
Name: 1040 Alt: bitcoin-21
Name: 1041 Alt: vip-tokens
Name: 1042 Alt: quebecoin
Name: 1043 Alt: cryptoworldx-token
Name: 1044 Alt: jewels
Name: 1045 Alt: bios-crypto
Name: 1046 Alt: dreamcoin
Name: 1047 Alt: uro
Name: 1048 Alt: flavorcoin
Name: 1049 Alt: destiny
Name: 1050 Alt: rsgpcoin
Name: 1051 Alt: plncoin
Name: 1052 Alt: billarycoin
Name: 1053 Alt: beavercoin
Name: 1054 Alt: zayedcoin
Name: 1055 Alt: dappster
Name: 1056 Alt: dollar-online
Name: 1057 Alt: arbit
Name: 1058 Alt: jobsco

Name: 1337 Alt: first-bitcoin
Name: 1338 Alt: high-gain
Name: 1339 Alt: aseancoin
Name: 1340 Alt: goldmaxcoin
Name: 1341 Alt: happy-creator-coin
Name: 1342 Alt: darklisk
Name: 1343 Alt: peoplecoin
Name: 1344 Alt: regacoin
Name: 1345 Alt: operand
Name: 1346 Alt: the-vegan-initiative
Name: 1347 Alt: hyper-tv
Name: 1348 Alt: teamup
Name: 1349 Alt: eusd
Name: 1350 Alt: digital-bullion-gold
Name: 1351 Alt: paypeer
Name: 1352 Alt: cheapcoin
Name: 1353 Alt: swapcoin
Name: 1354 Alt: tattoocoin-limited
Name: 1355 Alt: yescoin
Name: 1356 Alt: stex
Name: 1357 Alt: wa-space
Name: 1358 Alt: mavro
Name: 1359 Alt: international-diamond
Name: 1360 Alt: kashhcoin
Name: 1361 Alt: facecoin
Name: 1362 Alt: lazaruscoin
Name: 1363 Alt: cashme
Name: 1364 Alt: betacoin
Name: 1365 Alt: pinkdog
Name: 1366 Alt: utacoin
Name: 1367 Alt: president-johnson
Name: 1368 Alt: snakeeyes
Name: 1369 Alt: internet-of-things
Name: 1370 Alt: thecreed
Name: 1371 Alt: pokecoin
Name: 1372 Alt: yellow-token
Name: 1373 Alt: teslac